In [1]:
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd
%matplotlib inline
import os
import random
import itertools
import hickle as hkl

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
def reconstruct_images(plot_id):
    '''Takes a plot ID and subsets the input pd.DataFrame to that plot ID
       returns a (14, 14) array-like list with binary labels
       
        Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''
    #print(df.head(2))
    subs = df[df['PLOT_FNAME'] == plot_id]
    print(subs.shape)
    rows = []
    lats = reversed(sorted(subs['LAT'].unique()))
    for i, val in enumerate(lats):
        subs_lat = subs[subs['LAT'] == val]
        subs_lat = subs_lat.sort_values('LON', axis = 0)
        if 'PLANTATION' in df.columns:
            rows.append(list(subs_lat['PLANTATION']))
        elif 'SYSTEM' in df.columns:
            for i, val in subs_lat.iterrows():
                if val['SYSTEM'] == 'Not plantation':
                    rows.append(0)
                if val['SYSTEM'] == 'Monoculture':
                    rows.append(1)
                if val['SYSTEM'] == 'Agroforestry':
                    rows.append(2)
                if val['SYSTEM'] == 'Unknown':
                    rows.append(255)

    return rows

def load_csv(data_location, output_folder):
    df = pd.read_csv(data_location, encoding = "ISO-8859-1")
    df = df.rename(columns={df.columns[0]: 'plotid'})
    df.columns = [x.upper() for x in df.columns]
    print(df.head(5))
    #print(df['SYSTEM'].unique())
    columns = ['PLOTID', 'SAMPLEID', 'LON', 'LAT', 'PLANTATION', 'PLOT_FNAME', 'SYSTEM']
    for column in df.columns:
        if column not in columns:
            df = df.drop(column, axis = 1)
    print(df.head(5))
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_FNAME'].unique())
    print(plot_ids)
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    return df, plot_ids, existing

In [3]:

import re 
import pandas as pd
import os

CSV_PATH = 'train-csv/'
"""
cols_to_keep = ['PLOT_ID', 'SAMPLE_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES', 'USER_ID',
       'COLLECTION_TIME', 'ANALYSIS_DURATION', 'TREE', 'plotid', 'sampleid']

csvs = [x for x in sorted(os.listdir(CSV_PATH)) if "15" in x]
csvs = [x for x in csvs if ".csv" in x]


dfs = []
for i in csvs:
    df = pd.read_csv(CSV_PATH + i, encoding = "ISO-8859-1")
    df.columns = [re.sub(r'\W+', '', x) for x in df.columns]
    df.rename(columns={'ïplotid':'plotid'}, inplace=True)
    df.columns = [x.upper() for x in df.columns]
    df.columns = ['PLOT_ID' if x == 'PLOTID' else x for x in df.columns]
    df.columns = ['SAMPLE_ID' if x == 'SAMPLEID' else x for x in df.columns]
    df = df.rename(columns={df.columns[0]: 'PLOT_ID'})
    df = df.reset_index()
    df['PLOT_FNAME'] = '0'

    print(f"{str(i[-6:-4]).zfill(2)}")
    
    if len(df) > 0:
        counter = 0
        plot_ids = []
        for index, row in df.iterrows():
            if row['PLOT_ID'] not in plot_ids:
                plot_ids.append(row['PLOT_ID'])
                counter += 1
            df['PLOT_FNAME'][index] = f"{str(i[-6:-4]).zfill(2)}{str(counter).zfill(3)}"
        print("WRITING CSV")
        df.to_csv(csv_path + i, index = False)
"""

'\ncols_to_keep = [\'PLOT_ID\', \'SAMPLE_ID\', \'LON\', \'LAT\', \'FLAGGED\', \'ANALYSES\', \'USER_ID\',\n       \'COLLECTION_TIME\', \'ANALYSIS_DURATION\', \'TREE\', \'plotid\', \'sampleid\']\n\ncsvs = [x for x in sorted(os.listdir(CSV_PATH)) if "15" in x]\ncsvs = [x for x in csvs if ".csv" in x]\n\n\ndfs = []\nfor i in csvs:\n    df = pd.read_csv(CSV_PATH + i, encoding = "ISO-8859-1")\n    df.columns = [re.sub(r\'\\W+\', \'\', x) for x in df.columns]\n    df.rename(columns={\'ïplotid\':\'plotid\'}, inplace=True)\n    df.columns = [x.upper() for x in df.columns]\n    df.columns = [\'PLOT_ID\' if x == \'PLOTID\' else x for x in df.columns]\n    df.columns = [\'SAMPLE_ID\' if x == \'SAMPLEID\' else x for x in df.columns]\n    df = df.rename(columns={df.columns[0]: \'PLOT_ID\'})\n    df = df.reset_index()\n    df[\'PLOT_FNAME\'] = \'0\'\n\n    print(f"{str(i[-6:-4]).zfill(2)}")\n    \n    if len(df) > 0:\n        counter = 0\n        plot_ids = []\n        for index, row in df.iterrows()

In [4]:
cols_to_keep = ['PLOT_ID', 'SAMPLE_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES', 'USER_ID',
       'COLLECTION_TIME', 'ANALYSIS_DURATION', 'TREE', 'plotid', 'sampleid', 'PLOT_FNAME', 'SYSTEM']

csvs = [x for x in sorted(os.listdir(CSV_PATH))]
csvs = [x for x in csvs if 'v20' in x]

OUTPUT_FOLDER = 'train-labels/'


dfs = []
for i in csvs:
    
    df, plot_ids, existing = load_csv(CSV_PATH + i, OUTPUT_FOLDER)
    df.columns = [x.upper() for x in df.columns]
    
    df.columns = ['PLOT_ID' if x == 'PLOTID' else x for x in df.columns]
    df.columns = ['SAMPLE_ID' if x == 'SAMPLEID' else x for x in df.columns]

    for i in plot_ids:
        plot = reconstruct_images(i)
        plot = np.array(plot)
        print(plot.shape)
        np.save(f"train-labels/{str(i).zfill(5)}.npy", plot)

plot_ids = sorted(df['PLOT_FNAME'].unique())
plot_ids_loaded = plot_ids

print(f"There are {len(plot_ids)} plots")

   PLOTID  PLOT_ID  SAMPLE_ID       LON       LAT                  EMAIL  \
0       0        0       -195 -0.894849  6.116158  jessica.ertel@wri.org   
1       1        0       -194 -0.894849  6.116247  jessica.ertel@wri.org   
2       2        0       -193 -0.894849  6.116336  jessica.ertel@wri.org   
3       3        0       -192 -0.894849  6.116426  jessica.ertel@wri.org   
4       4        0       -191 -0.894849  6.116515  jessica.ertel@wri.org   

   FLAGGED   COLLECTION_TIME ANALYSIS_DURATION     IMAGERY_TITLE  \
0    False  2023-06-28 19:26         98.8 secs  Mapbox Satellite   
1    False  2023-06-28 19:26         98.8 secs  Mapbox Satellite   
2    False  2023-06-28 19:26         98.8 secs  Mapbox Satellite   
3    False  2023-06-28 19:26         98.8 secs  Mapbox Satellite   
4    False  2023-06-28 19:26         98.8 secs  Mapbox Satellite   

   IMAGERY_ATTRIBUTIONS                                  SAMPLE_GEOM   SYSTEM  \
0                   NaN  POINT(-0.894848875934758 6.1